# Calculadora muestral

In [4]:
# Calculadora de Tamaño Muestral y Estimación de Costos

import dash
from dash import dcc, html, Output, Input, State
import plotly.graph_objs as go
import numpy as np
import pandas as pd
from scipy.stats import norm
import geopandas as gpd
import dash_leaflet as dl
import dash_table
import dash_bootstrap_components as dbc

# Inicializar la aplicación
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
server = app.server

# Cargar datos espaciales
region = gpd.read_file("D:/muestreoasu/muestrageoref/VIVIENDAS_000.shp")
barrios = gpd.read_file("D:/muestreoasu/muestrageoref/Barrios_Localidades_Paraguay_Codigos_DGEEC.csv")


# Definir CRS si es necesario
if region.crs is None:
    region.crs = 'EPSG:32721'

if barrios.crs is None:
    barrios.crs = 'EPSG:32721'

region = region.to_crs(epsg=4326)
barrios = barrios.to_crs(epsg=4326)

# Estilos personalizados
header_style = {'backgroundColor': '#0d47a1', 'color': 'white', 'padding': '10px', 'textAlign': 'center', 'borderRadius': '10px'}
footer_style = {'backgroundColor': '#1565c0', 'color': 'white', 'padding': '10px', 'textAlign': 'center', 'borderRadius': '10px'}
panel_body_style = {'backgroundColor': '#e3f2fd', 'borderRadius': '10px', 'padding': '20px', 'width': '100%'}
formulas_style = {'backgroundColor': '#bbdefb', 'padding': '20px', 'borderRadius': '10px', 'marginTop': '20px'}

# Definir la interfaz de usuario
app.layout = html.Div([
    # Encabezado
    html.Div([
        html.H1("Calculadora de Tamaño Muestral y Estimación de Costos"),
        html.H5("Advertencia: Este es un ejemplo diseñado únicamente para ilustrar el cálculo de muestras totalmente aleatorias. Asegúrese de que sus datos cumplan con los supuestos necesarios antes de utilizar este método en la práctica."),
        html.H5("Este ejemplo es válido solo para unidades de viviendas en Asunción")
    ], style=header_style),
    
    # Pestañas
    dcc.Tabs([
        dcc.Tab(label='Calculadora', children=[
            html.Div([
                # Columna izquierda (entradas)
                html.Div([
                    html.Div([
                        html.H3("Parámetros de Entrada"),
                        html.Div([
                            html.Label("Tamaño de la Población:"),
                            dcc.Slider(
                                id='poblacion',
                                min=1000,
                                max=200000,
                                step=1000,
                                value=100000,
                                marks={i: str(i) for i in range(1000, 200001, 50000)},
                                tooltip={"placement": "bottom", "always_visible": True}
                            )
                        ], style={'marginBottom': '20px'}),
                        
                        html.Div([
                            html.Label("Nivel de Confianza (%):"),
                            dcc.Slider(
                                id='confianza',
                                min=50,
                                max=99.9,
                                step=0.1,
                                value=95,
                                marks={i: str(i) for i in range(50, 100, 10)},
                                tooltip={"placement": "bottom", "always_visible": True}
                            )
                        ], style={'marginBottom': '20px'}),
                        
                        html.Div([
                            html.Label("Margen de Error (%):"),
                            dcc.Slider(
                                id='error',
                                min=0.1,
                                max=20,
                                step=0.1,
                                value=5,
                                marks={i: str(i) for i in range(0, 21, 5)},
                                tooltip={"placement": "bottom", "always_visible": True}
                            )
                        ], style={'marginBottom': '20px'}),
                        
                        html.Div([
                            html.Label("Nivel de No Respuesta (%):"),
                            dcc.Slider(
                                id='no_respuesta',
                                min=0,
                                max=50,
                                step=1,
                                value=10,
                                marks={i: str(i) for i in range(0, 51, 10)},
                                tooltip={"placement": "bottom", "always_visible": True}
                            )
                        ], style={'marginBottom': '20px'}),
                        
                        html.Div([
                            html.Label("Costo por Encuesta (en Gs):"),
                            dcc.Input(
                                id='costo',
                                type='number',
                                value=50000,
                                min=1000,
                                style={'width': '100%'}
                            )
                        ], style={'marginBottom': '20px'}),
                        
                        html.Button('Recalcular', id='recalcular', n_clicks=0)
                    ], style=panel_body_style),
                    
                    html.Div([
                        html.H3("Distribución Normal"),
                        dcc.Graph(id='normal_plot')
                    ], style=panel_body_style)
                ], className='col-md-6'),
                
                # Columna derecha (salidas)
                html.Div([
                    html.Div([
                        html.H3("Resultados del Cálculo"),
                        html.Div(id='resultado', style={'whiteSpace': 'pre-wrap'}),
                        html.Div(id='costo_total', style={'whiteSpace': 'pre-wrap'})
                    ], style=panel_body_style),
                    
                    html.Div([
                        dl.Map(center=[-25.2826, -57.6359], zoom=13, children=[
                            dl.TileLayer(),
                            dl.LayerGroup(id='mapa')
                        ], style={'width': '100%', 'height': '600px'})
                    ], style={'marginTop': '20px'}),
                    
                    html.Div([
                        html.H3("Tabla de Puntos Muestrales Seleccionados"),
                        dash_table.DataTable(
                            id='tabla_puntos',
                            columns=[{'name': i, 'id': i} for i in ['ID', 'Barrio', 'lon', 'lat']],
                            data=[],
                            page_size=10
                        )
                    ], style=panel_body_style)
                ], className='col-md-6')
            ], className='row'),
            
            # Fórmulas
            html.Div([
                html.H3("Fórmulas Utilizadas"),
                html.P("Tamaño de muestra inicial (sin ajuste): n0 = (Z^2 * p * (1 - p)) / e^2"),
                html.P("Tamaño de muestra ajustado para poblaciones finitas: n = n0 / (1 + ((n0 - 1) / N))"),
                html.P("Tamaño de muestra final ajustado por no respuesta: n_final = n / (1 - NR)"),
                html.P("NR: Nivel de no respuesta, Z: Valor crítico para el nivel de confianza, e: Margen de error, p: Proporción esperada, N: Tamaño de la población")
            ], style=formulas_style)
        ]),
        dcc.Tab(label='Artículo', children=[
            html.Div([
                html.H3("Artículo"),
                dcc.Markdown('''
                # Artículo

                (Aquí iría el contenido del artículo)
                ''')
            ], style=panel_body_style)
        ])
    ]),
    
    # Pie de página
    html.Div([
        html.P("Diego Bernardo Meza - dmeza.py@gmail.com - 0971 100835"),
        html.P("Fuente: Geoportal del INE")
    ], style=footer_style)
], className='container-fluid')

# Callbacks

# Callback para actualizar resultados y gráfico
@app.callback(
    [Output('resultado', 'children'),
     Output('costo_total', 'children'),
     Output('normal_plot', 'figure')],
    [Input('recalcular', 'n_clicks')],
    [State('poblacion', 'value'),
     State('confianza', 'value'),
     State('error', 'value'),
     State('no_respuesta', 'value'),
     State('costo', 'value')]
)
def update_outputs(n_clicks, poblacion, confianza, error, no_respuesta, costo):
    # Realizar cálculos
    N = poblacion
    Z = norm.ppf(1 - (1 - (confianza / 100)) / 2)
    p = 0.5  # Suponiendo máxima variabilidad
    e = error / 100
    NR = 1 - (no_respuesta / 100)
    
    n0 = (Z ** 2 * p * (1 - p)) / (e ** 2)
    n = n0 / (1 + ((n0 - 1) / N))
    n = np.ceil(n)
    
    n_ajustado = np.ceil(n / NR)
    
    costo_total = n_ajustado * costo
    
    resultado_texto = f"El tamaño de muestra ajustado por no respuesta es: {int(n_ajustado)} individuos."
    costo_total_texto = f"El costo total estimado es: {int(costo_total):,} Gs"
    
    # Generar gráfico de distribución normal
    alpha = 1 - (confianza / 100)
    z_alpha = norm.ppf(1 - alpha / 2)
    
    SE = e / z_alpha
    
    x = np.linspace(-4 * SE, 4 * SE, 1000)
    y = norm.pdf(x, loc=0, scale=SE)
    
    lower_limit = -e
    upper_limit = e
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', line=dict(color='blue'), name='Distribución'))
    
    # Área del intervalo de confianza
    mask_confidence = (x >= lower_limit) & (x <= upper_limit)
    fig.add_trace(go.Scatter(
        x=np.concatenate([x[mask_confidence], x[mask_confidence][::-1]]),
        y=np.concatenate([y[mask_confidence], np.zeros_like(y[mask_confidence])]),
        fill='toself',
        fillcolor='rgba(0, 255, 0, 0.3)',
        line=dict(color='rgba(0,0,0,0)'),
        hoverinfo='skip',
        showlegend=False
    ))
    
    # Zonas de rechazo
    mask_rejection = (x < lower_limit) | (x > upper_limit)
    fig.add_trace(go.Scatter(
        x=np.concatenate([x[mask_rejection], x[mask_rejection][::-1]]),
        y=np.concatenate([y[mask_rejection], np.zeros_like(y[mask_rejection])]),
        fill='toself',
        fillcolor='rgba(255, 0, 0, 0.3)',
        line=dict(color='rgba(0,0,0,0)'),
        hoverinfo='skip',
        showlegend=False
    ))
    
    fig.update_layout(
        title=f"Distribución Normal con Nivel de Confianza: {confianza}% y Margen de Error: {e * 100}%",
        xaxis_title='Valor',
        yaxis_title='Densidad',
        showlegend=False
    )
    
    return resultado_texto, costo_total_texto, fig

# Callback para actualizar mapa y tabla
@app.callback(
    [Output('mapa', 'children'),
     Output('tabla_puntos', 'data')],
    [Input('recalcular', 'n_clicks')],
    [State('poblacion', 'value'),
     State('confianza', 'value'),
     State('error', 'value'),
     State('no_respuesta', 'value'),
     State('costo', 'value')]
)
def update_map_and_table(n_clicks, poblacion, confianza, error, no_respuesta, costo):
    # Cálculos previos
    N = poblacion
    Z = norm.ppf(1 - (1 - (confianza / 100)) / 2)
    p = 0.5
    e = error / 100
    NR = 1 - (no_respuesta / 100)
    
    n0 = (Z ** 2 * p * (1 - p)) / (e ** 2)
    n = n0 / (1 + ((n0 - 1) / N))
    n = np.ceil(n)
    
    n_ajustado = np.ceil(n / NR)
    n_ajustado = int(n_ajustado)
    
    # Muestreo de puntos
    if n_ajustado > len(region):
        n_ajustado = len(region)
    
    puntos_muestrales = region.sample(n=n_ajustado)
    puntos_muestrales.reset_index(drop=True, inplace=True)
    
    puntos_data = pd.DataFrame({
        'ID': puntos_muestrales.index + 1,
        'Barrio': puntos_muestrales['BARLO_DESC'].values,
        'lon': puntos_muestrales.geometry.x,
        'lat': puntos_muestrales.geometry.y
    })
    
    # Crear marcadores para el mapa
    markers = []
    for idx, row in puntos_data.iterrows():
        marker = dl.Marker(position=[row['lat'], row['lon']],
                           children=[
                               dl.Tooltip(row['Barrio']),
                               dl.Popup([
                                   html.H4(f"Barrio: {row['Barrio']}"),
                                   html.P(f"ID: {row['ID']}")
                               ])
                           ])
        markers.append(marker)
    
    mapa_children = markers
    
    # Datos para la tabla
    data_table = puntos_data.to_dict('records')
    
    return mapa_children, data_table

if __name__ == '__main__':
    app.run_server(debug=True)
